In [1]:
### https://www.youtube.com/watch?v=xdq6Gz33khQ

In [1]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [2]:
client_id='a2db305d6a8a4ebd8191989f7f400119'
client_secret = 'b0f792e64a1d4572906992c6148d319c'

# Create class functions 

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_status = True     # True if not expired and False if expiried
    client_id = None 
    client_secret = None 
    token_url= "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs): 
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_creditials(self): 
        """
            reuturns a base 64 encoded string././
        """
        self.client_id = client_id
        self.client_secret = client_secret
        
        if client_secret == None or client_id == None : 
            raise Exception("You must set client id and client sceret") 
        client_creds= f'{client_id}:{client_secret}'
        client_creds_b64=base64.b64encode(client_creds.encode())
        
        return client_creds_b64
    
    def get_token_headers(self): 
        client_creds_b64 = self.get_client_creditials()
        return { 
                "Authorization" : f'Basic {client_creds_b64.decode()}'  # <base64 encoded client_id:client_secret > 
                }
    
    def get_token_data(self):
        return {  
                'grant_type' : 'client_credentials'
                }
    
    def get_resource_header(self):
        access_token=self.get_access_token() 
        headers={
            "Authorization" : f"Bearer {access_token}"   
            }
        return headers
    
    def get_resource(self,lookup_id,resource_type='albums',version='v1'):
        endpoint=f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers=self.get_resource_header()
        r=requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            return {'No results'}
        return r.json()
    
#     def get_album(self,_id):
#         base_url='https://api.spotify.com/'
#         endpoint=f'{base_url}/v1/album/{_id}'
#         pass
    def get_album(self,_id): 
        return self.get_resource(_id,resource_type='albums')
    
    def get_artist(self,_id):
            return self.get_resource(_id,resource_type='artists')
        
#     def get_atrist(seldf,_id):
#         base_url='https://api.spotify.com/'
#         endpoint=f'{base_url}/v1/artist/{_id}'
#         pass

    def get_playlist(self,_id):
        access_token = self.get_access_token()
        playlist_url = f'https://api.spotify.com/v1/playlists/{_id}'
        headers={
                    "Authorization" : f"Bearer {access_token}"   
                    }

        response=requests.get(playlist_url, headers=headers)

        return response #self.get_resource(_id,resource_type='playlist')
    
    
    
    def get_access_token(self):
#         auth_done=self.perform_auth()
#         if not auth_done:
#             raise Exception("Authentication Failed")
        token=self.access_token
        expiry_time=self.access_token_expires
        current_time=datetime.datetime.now() 
        if expiry_time < current_time:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    

        
        
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers() 
        
        r=requests.post(token_url,data=token_data, headers=token_headers) 
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            raise Exception("Could not authenticate client")
        
        data=r.json()
        current_datetime=datetime.datetime.now() 
        access_token=data['access_token']
        expires_in=data['expires_in'] # seconds
        expires_time_left = current_datetime + datetime.timedelta(seconds=expires_in)
        self.access_token=access_token
        self.access_token_expires = expires_time_left
        self.access_token_expires_status = expires_time_left < current_datetime
        
        return True
    
    def base_search(self,query_params):
        access_token=self.get_access_token() 
        headers=self.get_resource_header()
        endpoint='https://api.spotify.com/v1/search'
        lookup_url=f'{endpoint}?{query_params}'
        r=requests.get(lookup_url ,headers=headers)
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            return {'No results'}
        return r.json()
    
    def search(self, query=None,search_type='artists'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query,dict):
            query=" ".join([f'{k}:{v}' for k,v in query.items()])
        query_params=urlencode({"q": query, "type" : search_type.lower()})
        return self.base_search(query_params)

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [18]:
xx=spotify.search("yellow submarine", search_type="Track")

In [77]:
access_token = spotify.get_access_token()

In [55]:
spotify.perform_auth()

True

In [63]:
#access_token = 'BQA3nc5xl9pdYeZ2X9wasbSosvsz_AMnJ1s8_KcF4TenPwYmXtiCM-K_NCytaMRZNMkTU9OR_XnV4ONZRjENzS3O9nyl4Uoa6SNvNKw2gn-sIAQNMAcsU6RO9E5smfIKkwQT-XxrsODMhiAxxYyjjo3TpXizm7CIz-n7cpTFYraY'

In [78]:
# playlist_url = 'https://api.spotify.com/v1/playlists/4GBXXvhMRpTpIxw6ekgRzM'
# headers={
#             "Authorization" : f"Bearer {access_token}"   
#             }
    

# response=requests.get(playlist_url, headers=headers)

In [80]:
# response.json()['tracks']['items'][0]['track']['name']

"Heaven Knows I'm Miserable Now - 2011 Remaster"

In [97]:
spotify.get_playlist('4GBXXvhMRpTpIxw6ekgRzM').json()['tracks']['items'][0]['track']['external_urls']['spotify']   #['tracks']['items'][0]['track']['name']

'https://open.spotify.com/track/1H5N26VqHR4JhuaRKY2I0u'

In [60]:
spotify.get_artist("3tJoFztHeIJkJWMrx0td2f")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3tJoFztHeIJkJWMrx0td2f'},
 'followers': {'href': None, 'total': 2301830},
 'genres': ['memphis hip hop',
  'rap',
  'southern hip hop',
  'tennessee hip hop',
  'trap'],
 'href': 'https://api.spotify.com/v1/artists/3tJoFztHeIJkJWMrx0td2f',
 'id': '3tJoFztHeIJkJWMrx0td2f',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb9d566d7d5ca673f9af94f4ce',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051749d566d7d5ca673f9af94f4ce',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1789d566d7d5ca673f9af94f4ce',
   'width': 160}],
 'name': 'Moneybagg Yo',
 'popularity': 81,
 'type': 'artist',
 'uri': 'spotify:artist:3tJoFztHeIJkJWMrx0td2f'}

In [61]:
spotify.get_album("7vEJAtP3KgKSpOHVgwm3Eh")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'},
   'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2',
   'id': '3WrFJ7ztbogyGnTHbHJFl2',
   'name': 'The Beatles',
   'type': 'artist',
   'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
  'KH'

In [6]:
import streamlit as st
import streamlit.components.v1 as components

st.title('hello world')

def select_and_play(feeling):
    happy = 'https://open.spotify.com/playlist/556ICk4gRzDknRfWGeQ3x1?si=317b09e2dc4b45f3'
    sad = 'https://open.spotify.com/playlist/0dRxDrR1PfZMlVbfnuBRbR?si=1c1095aff5cf4b24'
    anger = 'https://open.spotify.com/playlist/7FjP7MbRgFYFdv5avuhiBI?si=4289f7bbaa1243af'
    enthusiasm = 'https://open.spotify.com/playlist/4JsAKbWk4AoBcfpSs2afOM?si=d578284db05845ca'
    empty = 'https://open.spotify.com/playlist/77OpFLSdLy3nC9huQIXlxk?si=205fa8c2b1d24e8d'
    boredom = 'https://open.spotify.com/playlist/2rA0wLILuvNuLhAacn4kth?si=9ec06d60f5b14c53'
    worry = 'https://open.spotify.com/playlist/5Dt93qIXccZvZbU6r3oIbs?si=061a1e6b521b422f'
    love = 'https://open.spotify.com/playlist/73KuPUAtOecLDAetRn80TW?si=dda2764abcde4498'
    surprise = 'https://open.spotify.com/playlist/0BaRZECQEqp4zDd0Njzlj1?si=d943c1a27fec45df'
    fun = 'https://open.spotify.com/playlist/7HwdXmzNKXBzTAisOKYVsJ?si=6f73860860db49d1'
    relief = 'https://open.spotify.com/playlist/2zaAFRdI6lEaX8Esc11XPZ?si=b4ea70e95eca4506'
    hate = 'https://open.spotify.com/playlist/6vdOF3ZRqiYimqEm4lk98V?si=eb11df3e48b44e4e'
    neutral = 'https://open.spotify.com/playlist/5pSdjjPHbXpbqFJGf31Ksn?si=8a2e9c4bf0e04ef5'

    d= {'happy':happy, 'sad':sad, 'worry':worry, 'love':love, 
        'anger':anger, 'fun':fun, 'relief': relief, 'empty':empty,
        'hate':hate, 'enthusiasm':enthusiasm, 'surprise':surprise,
        'boredom':boredom, 'neutral':neutral
       }
    
    mood = d[feeling]

    return components.html(
            f'<iframe src={mood} width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>',
            height=600
            )

# components.html(
#     '''
#         <iframe src="https://open.spotify.com/embed/playlist/7xOHp3ZlSBJNJOgsQwF85S" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>
#     ''',
#     height=600
# )

2021-09-08 11:46:10.205 
  command:

    streamlit run /home/iases/.pyenv/versions/3.8.6/envs/ser_proj/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [8]:
select_and_play('happy')

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)